In [1]:
import os
import pprint
import tempfile
import itertools

from typing import Dict, Text

import math
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-06-07 19:00:13.271942: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-07 19:00:13.271992: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Global vars

MAX_HISTORY = 7

# Load both dataset and convert to tfds object

In [3]:
def slices(features):
  for i in itertools.count():
    # For each feature take index `i`
    example = {name:values[i] for name, values in features.items()}
    yield example

## Load courses list
TODO: Course list disini sepertinya rada galengkap, token dari course" bangkit jadi nol semua

Output berupa objek MapDataset

In [4]:
courses = pd.read_csv("courses.csv")
unique_course_names = courses["name"].tolist()

courses_dict = {
  "course_name": np.array(unique_course_names)
}

for example in slices(courses_dict):
  for name, value in example.items():
    print(f"{name:19s}: {value}")
  break

tfds_courses = tf.data.Dataset.from_tensor_slices(courses_dict)
tfdsmap_courses = tfds_courses.map(lambda x: x['course_name'])
tfdsmap_courses

course_name        : Belajar Fundamental Aplikasi Android


2022-06-07 19:01:16.333706: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-07 19:01:16.333784: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-07 19:01:16.333842: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ratatoskr): /proc/driver/nvidia/version does not exist
2022-06-07 19:01:16.539292: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [5]:

def get_tokenizer_dict(train_list):
  return dict(zip(train_list, [i+1 for i in range(len(train_list))]))

get_tokenizer_dict(unique_course_names)

{'Belajar Fundamental Aplikasi Android': 1,
 'Belajar Membangun LINE Chatbot': 2,
 'Belajar Membuat Aplikasi Android untuk Pemula': 3,
 'Memulai Pemrograman Dengan Java': 4,
 'Memulai Pemrograman Dengan Kotlin': 5,
 'Menjadi Azure Cloud Developer': 6,
 'Memulai Pemrograman Dengan Python': 7,
 'Memulai Pemrograman Dengan C': 8,
 'Belajar Dasar Pemrograman Web': 9,
 'Menjadi Google Cloud Engineer': 10,
 'Belajar Dasar-Dasar Azure Cloud': 11,
 'Memulai Pemrograman Dengan Swift': 12,
 'Belajar Membangun LINE Front-end Framework (LIFF)': 13,
 'Belajar Membuat Aplikasi Flutter untuk Pemula': 14,
 'Belajar Fundamental Front-End Web Development': 15,
 'Menjadi Android Developer Expert': 16,
 'Belajar Prinsip Pemrograman SOLID': 17,
 'Belajar Membuat Aplikasi iOS untuk Pemula': 18,
 'Belajar Dasar Visualisasi Data': 19,
 'Belajar Machine Learning untuk Pemula': 20,
 'Belajar Pengembangan Machine Learning': 21,
 'Memulai Pemrograman Dengan Dart': 22,
 'Belajar Fundamental Aplikasi Flutter': 23,


In [6]:
# Load and clean data
useract = pd.read_csv("user_activities.csv")
useract = useract.drop(columns=['id'])
useract = useract.dropna(how='all')

def mergeall(rowdata):
    aggregated_datas = ",".join([str(s) for s in list(rowdata.values) if not pd.isna(s)])
    return aggregated_datas.split(",")

def mergecourse(rowdata):
    aggregated_datas = ",".join([str(s) for s in [rowdata.graduated_courses, rowdata.on_progress_courses] if not pd.isna(s)])
    return aggregated_datas.split(",")

merged_datas = list(useract.apply(mergeall, axis = 1))
merged_courses = list(useract.apply(mergecourse, axis = 1))

agumented_datas_dict = {
  'taken_courses': [],
  'recommendation': []
}

# for each merged course, 
#   for each course in merged course,
#     pick 1 as output, lainya jadi input
#     push as new entry
# TODO: 
#   This split is not too well-defined on merged courses with length longer than MAX_HISTORY.
#   This is because after permutating possible input, trailing courses are cut off at token padding anyway.

for i in range(len(merged_datas)):
  merged_data = merged_datas[i]
  merged_course = merged_courses[i]
    
  for course in merged_course:
    #Simpen dulu buat nanti kalo mau ditambahin ingfo non-courses
    #agumented_datas_dict['x'].append([data for data in merged_data if data != course])
    agumented_datas_dict['taken_courses'].append([data for data in merged_course if data != course])
    agumented_datas_dict['recommendation'].append(course)

for example in slices(agumented_datas_dict):
  for name, value in example.items():
    print(f"{name:19s}: {value}")
  break

taken_courses      : ['Bangkit General Assessment', 'Belajar Dasar Pemrograman Web', 'Belajar Fundamental Aplikasi Android']
recommendation     : Bangkit Android Assessment


## Load user activities list
User Activities merujuk pada semua course yang pernah dan sedang diambil. Asumsinya adalah kalau banyak user yang ambil bebarengan, maka seharusnya course" tsb berkaitan.

Augmentasi data dilakukan sbb:
```
Jika course yang pernah diambil adalah [a, b, c, d, e]
output akan diaugmentasi menjadi:
x             | y
--------------+----
[a, b, c, d]  | e
[a, b, c, e]  | d
[a, b, d, e]  | c
[a, c, d, e]  | b
[b, c, d, e]  | a

```

Output berupa objek MapDataset

In [7]:
# Tokenize manual karena usecasenya agak aneh

def get_tokenizer_dict(train_list):
  return dict(zip(train_list, [i+1 for i in range(len(train_list))]))

def pad_tokens(input_sequences, maxlen):
  return np.array(pad_sequences(input_sequences, maxlen=maxlen, padding='pre'))

def tokenize(tokenizer_dict, corpus, maxlen = 5):
  output = []
  for line in corpus:
    tokenizedline = [tokenizer_dict.get(entry, 0) for entry in line]
    output.append(tokenizedline)
    
  return pad_tokens(output, maxlen)


tokenizer_dict = get_tokenizer_dict(unique_course_names)
tokenized_datas = tokenize(tokenizer_dict, agumented_datas_dict["taken_courses"], MAX_HISTORY)
tokenized_datas_dict = {
  'taken_courses': tokenized_datas,
  'recommendation': agumented_datas_dict["recommendation"]
}

for example in slices(tokenized_datas_dict):
  for name, value in example.items():
    print(f"{name:19s}: {value}")
  break

tfds_tokenized_data = tf.data.Dataset.from_tensor_slices(tokenized_datas_dict)
tfdsmap_tokenized_data = tfds_tokenized_data.map(lambda x: {
  'taken_courses': x['taken_courses'],
  'recommendation': x['recommendation']
})
tfdsmap_tokenized_data

taken_courses      : [0 0 0 0 0 9 1]
recommendation     : Bangkit Android Assessment


<MapDataset element_spec={'taken_courses': TensorSpec(shape=(7,), dtype=tf.int32, name=None), 'recommendation': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [8]:
print(len(tfdsmap_courses))
print(len(tfdsmap_tokenized_data))

48
1719


In [33]:
tf.random.set_seed(42069)
shuffled = tfdsmap_tokenized_data.shuffle(len(tfdsmap_tokenized_data), seed=69420, reshuffle_each_iteration=False)

train_len = math.floor(len(tfdsmap_tokenized_data) * 0.9)
test_len = len(tfdsmap_tokenized_data) - train_len

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)

print(train, len(train))
print(test, len(test))

<TakeDataset element_spec={'taken_courses': TensorSpec(shape=(7,), dtype=tf.int32, name=None), 'recommendation': TensorSpec(shape=(), dtype=tf.string, name=None)}> 1547
<TakeDataset element_spec={'taken_courses': TensorSpec(shape=(7,), dtype=tf.int32, name=None), 'recommendation': TensorSpec(shape=(), dtype=tf.string, name=None)}> 172


In [55]:
embedding_dimension = 32

#https://www.tensorflow.org/recommenders/examples/multitask/

user_properties_model = tf.keras.Sequential([
  tf.keras.layers.Embedding(len(unique_course_names)+1, 32, input_length=MAX_HISTORY),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(150, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(embedding_dimension, activation='relu')
])

course_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=unique_course_names, mask_token=None),
  tf.keras.layers.Embedding(len(unique_course_names) + 1, embedding_dimension),
])

task = tfrs.tasks.Retrieval(
  metrics=tfrs.metrics.FactorizedTopK(
    candidates=tfdsmap_courses.batch(16).map(course_model)
  )
)

In [56]:
class Model(tfrs.Model):
  def __init__(self, user_model, course_model):
    super().__init__()
    self.course_model: tf.keras.Model = course_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[tf.Tensor, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["taken_courses"])

    # And pick out the course features and pass them into the course model,
    # getting embeddings back.
    positive_course_embeddings = self.course_model(features["recommendation"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_course_embeddings)

In [57]:
model = Model(user_properties_model, course_model)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))

cached_train = train.shuffle(len(tfdsmap_tokenized_data)).batch(400).cache()
cached_test = test.batch(40).cache()

user_properties_model.summary()
course_model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 7, 32)             1568      
                                                                 
 flatten_5 (Flatten)         (None, 224)               0         
                                                                 
 dense_12 (Dense)            (None, 150)               33750     
                                                                 
 dense_13 (Dense)            (None, 100)               15100     
                                                                 
 dense_14 (Dense)            (None, 32)                3232      
                                                                 
Total params: 53,650
Trainable params: 53,650
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_11"
__________________________

In [58]:
model.fit(cached_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 2s 68ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0129 - factorized_top_k/top_5_categorical_accuracy: 0.1067 - factorized_top_k/top_10_categorical_accuracy: 0.1810 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 2973.8888 - regularization_loss: 0.0000e+00 - total_loss: 2973.8888
Epoch 2/5
4/4 [==============================] - 0s 66ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0129 - factorized_top_k/top_5_categorical_accuracy: 0.2043 - factorized_top_k/top_10_categorical_accuracy: 0.2954 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 2429.0603 - regularization_loss: 0.0000e+00 - total_loss: 2429.0603
Epoch 3/5
4/4 [==============================] - 0s 66ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2967 - factorized_top_k/top_5_categorical_accuracy: 0.4163 - fact

In [59]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 66ms/step - factorized_top_k/top_1_categorical_accuracy: 0.9884 - factorized_top_k/top_5_categorical_accuracy: 0.9884 - factorized_top_k/top_10_categorical_accuracy: 0.9942 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 109.5345 - regularization_loss: 0.0000e+00 - total_loss: 109.5345


{'factorized_top_k/top_1_categorical_accuracy': 0.9883720874786377,
 'factorized_top_k/top_5_categorical_accuracy': 0.9883720874786377,
 'factorized_top_k/top_10_categorical_accuracy': 0.9941860437393188,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'loss': 29.818880081176758,
 'regularization_loss': 0,
 'total_loss': 29.818880081176758}

# Making predicitons


In [60]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

# recommends courses out of the entire courses dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((
    tfdsmap_courses.batch(10), 
    tfdsmap_courses.batch(10).map(model.course_model)
  ))
)

In [61]:
taken_courses = [

]
inputdata = tokenize(tokenizer_dict, [taken_courses], MAX_HISTORY)
print(inputdata)
_, course_names = index(inputdata)
course_names

[[0 0 0 0 0 0 0]]


<tf.Tensor: shape=(1, 10), dtype=string, numpy=
array([[b'Belajar Fundamental Aplikasi Android',
        b'Belajar Membangun LINE Chatbot',
        b'Belajar Membuat Aplikasi Android untuk Pemula',
        b'Memulai Pemrograman Dengan Java',
        b'Memulai Pemrograman Dengan Kotlin',
        b'Menjadi Azure Cloud Developer',
        b'Memulai Pemrograman Dengan Python',
        b'Memulai Pemrograman Dengan C',
        b'Belajar Dasar Pemrograman Web',
        b'Menjadi Google Cloud Engineer']], dtype=object)>

In [69]:

# Get recommendations.
taken_courses = [
  "Belajar Dasar Pemrograman Web,Meniti Karier sebagai Software Developer",
  "Belajar Membuat Augmented Reality dengan Lens Studio",
  "Belajar Dasar Pemrograman JavaScript",
  "Belajar Fundamental Front-End Web Development",
  "Belajar Membuat Aplikasi Android untuk Pemula"
]
inputdata = tokenize(tokenizer_dict, [taken_courses], MAX_HISTORY)
print(inputdata)

_, course_names = index(inputdata)
print("Recommendation:")
for course_name in course_names[0, :4]:
  tf.print(course_name)

[[ 0  0  0 38 30 15  3]]
Recommendation:
Belajar Fundamental Aplikasi Android
Belajar Membangun LINE Chatbot
Belajar Membuat Aplikasi Android untuk Pemula
Memulai Pemrograman Dengan Java


In [70]:
tf.saved_model.save(index, "./saved_index")

INFO:tensorflow:Assets written to: ./saved_index/assets


INFO:tensorflow:Assets written to: ./saved_index/assets


In [72]:
# load it back; can also be done in TensorFlow Serving.
loaded = tf.saved_model.load("./saved_index")

# pass a user id in, get top predicted movie titles back.
scores, titles = loaded(inputdata)

_, course_names = loaded(inputdata)
print("Recommendation:")
for course_name in course_names[0, :4]:
  tf.print(course_name)

Recommendation:
Belajar Fundamental Aplikasi Android
Belajar Membangun LINE Chatbot
Belajar Membuat Aplikasi Android untuk Pemula
Memulai Pemrograman Dengan Java
